In [1]:
#https://sbert.net/
#https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2
#https://huggingface.co/openai/whisper-large-v3
#https://pypi.org/project/faiss-cpu/

In [2]:
import json

# Load the list from the JSON file
with open('speech_total.json', 'r') as file:
    speech_total = json.load(file)

len(speech_total)

828

In [3]:
# Setting up Groq API
import os
from dotenv import load_dotenv
from groq import Groq

load_dotenv()
client = Groq(api_key=os.getenv("GROQ_API"))

def get_groq_response(query):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": query,
            }
        ],
        model="mixtral-8x7b-32768",
        temperature=0.5,
        max_tokens=1024,
    )
    return chat_completion.choices[0].message.content

In [8]:
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import faiss
import pickle
import torch

# Step 1: Load the embedding model and create embeddings
embed_model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings
embeddings = embed_model.encode(speech_total, show_progress_bar=True)
print(f"Embeddings shape: {embeddings.shape}")

# Step 2: Create and save FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings.astype('float32'))

# Save the index and texts
faiss.write_index(index, "faiss_index.bin")
with open('original_texts.pkl', 'wb') as f:
    pickle.dump(speech_total, f)

# Step 3: Load the language model and tokenizer
model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Set the pad token to be the eos token
tokenizer.pad_token = tokenizer.eos_token

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Create a text generation pipeline
generator = pipeline('text-generation', model=model, tokenizer=tokenizer, device=0 if torch.cuda.is_available() else -1)

def retrieve_context(query, k=3):
    query_embedding = embed_model.encode([query])[0]
    D, I = index.search(np.array([query_embedding]).astype('float32'), k)
    retrieved_texts = [speech_total[i] for i in I[0]]
    return " ".join(retrieved_texts)

def generate_response(query, max_new_tokens=200):
    # Get factual response from Groq
    groq_response = get_groq_response(query)
    
    # Retrieve context from your existing data
    context = retrieve_context(query)
    
    # Combine Groq response with retrieved context
    combined_input = f"Context: {context}\n\nGroq Response: {groq_response}\n\nQuery: {query}\n\nResponse:"
    
    # Tokenize the input
    inputs = tokenizer(combined_input, return_tensors="pt", padding=True, truncation=True, max_length=512)
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)
    
    # Generate Elon-style response
    output = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_new_tokens=max_new_tokens,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id
    )
    
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    generated_text = response.split("Response:")[-1].strip()
    
    # Further "Elonify" the response
    return elonify(generated_text)

def elonify(content):
    prompt = f"""
    Rewrite the following content in Elon Musk's speaking style, based on the training data:

    {content}

    Elon Musk's response:
    """
    
    # Tokenize the input
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=512)
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)
    
    output = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_new_tokens=200,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id
    )
    
    return tokenizer.decode(output[0], skip_special_tokens=True).split("Elon Musk's response:")[-1].strip()
# Example usage
query = "What are Elon Musk's thoughts on AI?"
response = generate_response(query)
print(f"Query: {query}")
print(f"Response: {response}")

Batches:   0%|          | 0/26 [00:00<?, ?it/s]

Embeddings shape: (828, 384)
Query: What are Elon Musk's thoughts on AI?
Response: The problem is that we have to deal with large numbers of people. That is a problem that has to be solved. The answer is to not ask for a number of numbers. You have no control, you have only control to decide what to say. I say we do have a limit, so it's not a big deal. There are some people who have been saying I have asked for this limit and that's the problem with our machine. And it has very different consequences. As one person said, there are thousands of cases where I am not able understand what is right and what isn't right. Now we may just have the answer. So it may be wrong for us to ask a question that I can just say, 'Well what am I doing with this machine?' I mean, I don't know what the issue is. What is wrong is what I'm doing. In fact, what you are saying is, if you ask me, how can you understand that point of view


In [9]:
import random
from tqdm import tqdm

def evaluate_rag_system(num_samples=100):
    # Sample queries or create a test set
    test_queries = [
        "What does Elon Musk think about renewable energy?",
        "How does Elon Musk view the future of space exploration?",
        # Add more diverse queries here
    ]

    results = []
    for query in tqdm(random.sample(test_queries, min(num_samples, len(test_queries)))):
        response = generate_response(query)
        results.append({
            "query": query,
            "response": response,
            # You might add more metrics here, like response time, etc.
        })

    # Here you would typically add code to calculate metrics
    # such as relevance, coherence, factual accuracy, etc.
    # This often requires human evaluation or comparison against known ground truths

    return results

# Run evaluation
eval_results = evaluate_rag_system()
print(f"Evaluated {len(eval_results)} queries")
# Add code here to analyze and display results

100%|██████████| 2/2 [00:22<00:00, 11.39s/it]

Evaluated 2 queries


In [10]:
from IPython.display import clear_output

def interactive_qa():
    while True:
        query = input("Enter your question (or type 'exit' to quit): ")
        if query.lower() == 'exit':
            print("Thank you for using the Q&A system. Goodbye!")
            break
        
        if query.strip():
            print(f"\nQ: {query}")
            try:
                response = generate_response(query)
                print(f"\nA: {response}\n")
            except Exception as e:
                print(f"An error occurred: {str(e)}\n")
        else:
            print("Please enter a question.\n")
        
        # Optional: Uncomment the next line if you want to clear output after each Q&A
        # clear_output(wait=True)

print("The Q&A system is ready. Type your question and press Enter.")
interactive_qa()

The Q&A system is ready. Type your question and press Enter.

Q: what does spacex do? and whats the difference between jews and christians

A: Rewrite the following content in Elon Musk's speaking style, based on the training data:

    SpaceX is a private aerospace manufacturer and space transportation company founded by Elon Musk in 2002. Its mission is to reduce space transportation costs and enable the colonization of Mars. SpaceX has developed several launch vehicles and the Dragon spacecraft, which is flown into orbit by the Falcon launch vehicles. The Dragon spacecraft is used to deliver cargo, and in the future, crew members to the International Space Station (ISS).

Jews and Christians are two monotheistic religions that share some historical and theological connections but also have significant differences. Here are some of the main differences:

1. God: Both religions believe in one God, but Jews believe that God is a singular, indivisible entity, while Christians believe in